In [1]:
import tensorflow as tf
from src.custom.layers import LandmarkEmbedding
from src.custom.metrics import SparseLevenshtein
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2023-08-22 23:24:53.720390: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 23:24:54.290325: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
test_model = tf.keras.Sequential([tf.keras.Input(shape=(10,)), tf.keras.layers.Dense(10, activation="relu"), tf.keras.layers.Dense(3, activation="softmax")])

2023-08-22 23:24:54.824675: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-22 23:24:54.843890: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-22 23:24:54.844041: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
test_model.save("../models/test", save_format="tf")

INFO:tensorflow:Assets written to: ../models/test/assets


In [4]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(test_model)
# keras_model_converter = tf.lite.TFLiteConverter.from_saved_model("../models/prod_v1")
keras_model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
keras_model_converter.allow_custom_ops = True
tflite_model = keras_model_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp72hkzi6q/assets


2023-08-22 23:24:55.925317: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-08-22 23:24:55.925331: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-08-22 23:24:55.925706: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp72hkzi6q
2023-08-22 23:24:55.926092: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-08-22 23:24:55.926101: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp72hkzi6q
2023-08-22 23:24:55.927566: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-08-22 23:24:55.927852: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-08-22 23:24:55.955476: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmp72hkzi6q
2023-08

In [5]:
class TestModel(tf.keras.Model):
    def __init__(self, units, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.units = units
        self.dense = tf.keras.layers.Dense(units=16, activation="relu")
        self.concatenate = tf.keras.layers.Concatenate()
        self.classification = tf.keras.layers.Dense(units=units, activation="softmax")

    def call(self, inputs, training=False, mask=None):
        x1 = self.dense(inputs["source"])
        x2 = self.dense(inputs["target"])
        x = self.concatenate([x1, x2])
        x = self.classification(x)

        return x

    def generate(self, inputs):
        return self({"sourece":inputs, "target":inputs})

    def get_config(self):
        config = super().get_config()
        config.update({"units": self.units})
        return config

In [6]:
test_model_class = TestModel(units=10)

test_model_class.build({"source":(None, 1, 5), "target":(None, 1, 5)})

test_model_class.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                                metrics=["accuracy", SparseLevenshtein()])

x1 = tf.random.uniform(shape=(120, 1, 5), maxval=100, dtype=tf.int32)
x2 = tf.random.uniform(shape=(120, 1, 5), maxval=100, dtype=tf.int32)
y = tf.random.uniform(shape=(120, 1), maxval=10, dtype=tf.int32)

test_model_class.fit(x={"source":x1, "target":x2}, y=y)

2023-08-22 23:24:56.992343: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7efc6c014450 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-22 23:24:56.992361: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2023-08-22 23:24:57.099014: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-08-22 23:24:57.141293: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-22 23:24:57.179315: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4/4 [==============================] - 1s 5ms/step - loss: 48.6875 - accuracy: 0.0833 - sparse_levenshtein: inf


In [7]:
test_model_class.summary()

Model: "test_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             multiple                  96        
                                                                 
 concatenate (Concatenate)   multiple                  0         
                                                                 
 dense_3 (Dense)             multiple                  330       
                                                                 
Total params: 426
Trainable params: 426
Non-trainable params: 0
_________________________________________________________________


In [8]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(test_model_class)
keras_model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
keras_model_converter.allow_custom_ops = True
tflite_model = keras_model_converter.convert()

2023-08-22 23:24:57.538375: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'source' with dtype int32 and shape [?,1,5]
	 [[{{node source}}]]
2023-08-22 23:24:57.539429: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'target' with dtype int32 and shape [?,1,5]
	 [[{{node target}}]]
2023-08-22 23:24:57.559486: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'source' with dtype int32 and shape [?,1,5]
	 [[{{node source}}]]
2023-08-22 23:24

INFO:tensorflow:Assets written to: /tmp/tmpcsyha9ki/assets


2023-08-22 23:24:57.752366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,32]
	 [[{{node inputs}}]]
2023-08-22 23:24:57.790333: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_source' with dtype int32 and shape [?,1,5]
	 [[{{node serving_default_source}}]]
2023-08-22 23:24:57.790389: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_target' with dtype int32 and sha

In [9]:
from src.constants import FEATURE_COLUMNS
from src.data_utils.dataset import start_token_idx, end_token_idx
from src.data_utils.dataset import pre_process


class TFLiteModel(tf.Module):
    def __init__(self, model):
        super(TFLiteModel, self).__init__()
        self.target_start_token_idx = start_token_idx
        self.target_end_token_idx = end_token_idx
        # Load the feature generation and main models
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 1, 5], dtype=tf.int32, name='inputs')])
    def __call__(self, inputs, training=False):
        # Preprocess Data
        x = self.model({"source": inputs, "target": inputs})
        return {'outputs': x}

tflitemodel_base = TFLiteModel(test_model_class)

In [10]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflitemodel_base)
keras_model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
keras_model_converter.allow_custom_ops = True
tflite_model = keras_model_converter.convert()

2023-08-22 23:24:58.338663: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype int32 and shape [?,1,5]
	 [[{{node inputs}}]]
2023-08-22 23:24:58.387856: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'source' with dtype int32 and shape [?,1,5]
	 [[{{node source}}]]
2023-08-22 23:24:58.388874: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'target' with dtype int32 and shape [?,1,5]
	 [[{{node target}}]]
2023-08-22 23:24

INFO:tensorflow:Assets written to: /tmp/tmp7ua2tinm/assets


INFO:tensorflow:Assets written to: /tmp/tmp7ua2tinm/assets
2023-08-22 23:24:58.838599: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_inputs' with dtype int32 and shape [?,1,5]
	 [[{{node serving_default_inputs}}]]
2023-08-22 23:24:58.978438: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-08-22 23:24:58.978452: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-08-22 23:24:58.978569: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp7ua2tinm
2023-08-22 23:24:58.979726: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-08-22 23:24:58.979735: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (

In [11]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

interpreter.get_signature_list()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


{'serving_default': {'inputs': ['inputs'], 'outputs': ['outputs']}}

In [12]:
prediction_fn = interpreter.get_signature_runner("serving_default")

prediction_fn(inputs=x1)

{'outputs': array([[[8.23194087e-01, 0.00000000e+00, 1.50051856e-19, ...,
          4.31375824e-10, 4.73355776e-31, 1.83951354e-10]],
 
        [[1.74251831e-08, 8.42565153e-14, 1.59119625e-12, ...,
          9.27534711e-04, 8.61044407e-01, 1.08396366e-01]],
 
        [[9.15976584e-01, 0.00000000e+00, 7.77533620e-23, ...,
          1.80580562e-09, 2.60778304e-37, 6.51901064e-06]],
 
        ...,
 
        [[9.85538721e-01, 2.99478859e-34, 8.32775559e-19, ...,
          1.83338834e-05, 3.75000227e-29, 3.04326550e-05]],
 
        [[3.64003852e-02, 1.57013767e-36, 1.23427987e-11, ...,
          8.44432488e-02, 3.17403881e-11, 4.56606634e-02]],
 
        [[8.86248112e-01, 2.47107965e-30, 6.90677098e-08, ...,
          1.39337253e-05, 3.73529610e-11, 1.37974987e-09]]], dtype=float32)}

In [13]:
# No build

In [14]:
test_model_class = TestModel(units=10)

# test_model_class.build({"source":(None, 1, 5), "target":(None, 1, 5)})

test_model_class.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                                metrics=["accuracy", SparseLevenshtein()])

x1 = tf.random.uniform(shape=(120, 1, 5), maxval=100, dtype=tf.int32)
x2 = tf.random.uniform(shape=(120, 1, 5), maxval=100, dtype=tf.int32)
y = tf.random.uniform(shape=(120, 1), maxval=10, dtype=tf.int32)

test_model_class.fit(x={"source":x1, "target":x2}, y=y)

4/4 [==============================] - 1s 4ms/step - loss: 39.3638 - accuracy: 0.1167 - sparse_levenshtein: inf


In [15]:
test_model_class.summary()

Model: "test_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             multiple                  96        
                                                                 
 concatenate_1 (Concatenate)  multiple                 0         
                                                                 
 dense_5 (Dense)             multiple                  330       
                                                                 
Total params: 426
Trainable params: 426
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Covnert

keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(test_model_class)
keras_model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
keras_model_converter.allow_custom_ops = True
tflite_model = keras_model_converter.convert()

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None, 16), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c340190>, 139627446117232), {}).


2023-08-22 23:24:59.909873: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'source' with dtype int32 and shape [?,1,5]
	 [[{{node source}}]]
2023-08-22 23:24:59.911086: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'target' with dtype int32 and shape [?,1,5]
	 [[{{node target}}]]
2023-08-22 23:24:59.930772: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'source' with dtype int32 and shape [?,1,5]
	 [[{{node source}}]]
2023-08-22 23:24

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(16,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c342440>, 139627445638192), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(16,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c342440>, 139627445638192), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None, 10), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c36c250>, 139627446846640), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None, 10), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c36c250>, 139627446846640), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(10,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c36e050>, 139627445642432), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(10,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c36e050>, 139627445642432), {}).
2023-08-22 23:25:00.150346: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_source' with dtype int32 and shape [?,1,5]
	 [[{{node serving_default_source}}]]
2023-08-22 23:25:00.150437: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_target' with dtype int32 and shape [?,1,5]
	 [[{{node serving_default_target}}]]


INFO:tensorflow:Assets written to: /tmp/tmp4fdrmkma/assets


INFO:tensorflow:Assets written to: /tmp/tmp4fdrmkma/assets
2023-08-22 23:25:00.459004: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_source' with dtype int32 and shape [?,1,5]
	 [[{{node serving_default_source}}]]
2023-08-22 23:25:00.459073: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_target' with dtype int32 and shape [?,1,5]
	 [[{{node serving_default_target}}]]
2023-08-22 23:25:00.590609: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-08-22 23:25:00.590626: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:3

In [17]:
# WARNING:absl:Found untraced functions such as _update_step_xla while saving (showing 1 of 1). These functions will not be directly callable after loading.


In [18]:
# Wrapper

from src.constants import FEATURE_COLUMNS
from src.data_utils.dataset import start_token_idx, end_token_idx
from src.data_utils.dataset import pre_process


class TFLiteModel(tf.Module):
    def __init__(self, model):
        super(TFLiteModel, self).__init__()
        self.target_start_token_idx = start_token_idx
        self.target_end_token_idx = end_token_idx
        # Load the feature generation and main models
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 1, 5], dtype=tf.int32, name='inputs')])
    def __call__(self, inputs, training=False):
        # Preprocess Data
        x = self.model({"source": inputs, "target": inputs})
        return {'outputs': x}

tflitemodel_base = TFLiteModel(test_model_class)

In [19]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflitemodel_base)
keras_model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
keras_model_converter.allow_custom_ops = True
tflite_model = keras_model_converter.convert()

2023-08-22 23:25:00.793815: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype int32 and shape [?,1,5]
	 [[{{node inputs}}]]
2023-08-22 23:25:00.842132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'source' with dtype int32 and shape [?,1,5]
	 [[{{node source}}]]
2023-08-22 23:25:00.843162: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'target' with dtype int32 and shape [?,1,5]
	 [[{{node target}}]]
2023-08-22 23:25

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None, 16), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c340190>, 139627446117232), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None, 16), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c340190>, 139627446117232), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(16,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c342440>, 139627445638192), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(16,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c342440>, 139627445638192), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None, 10), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c36c250>, 139627446846640), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(10,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c36e050>, 139627445642432), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(10,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efd8c36e050>, 139627445642432), {}).
2023-08-22 23:25:01.084983: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_inputs' with dtype int32 and shape [?,1,5]
	 [[{{node serving_default_inputs}}]]


INFO:tensorflow:Assets written to: /tmp/tmpa_q_2qtl/assets


INFO:tensorflow:Assets written to: /tmp/tmpa_q_2qtl/assets
2023-08-22 23:25:01.301923: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_inputs' with dtype int32 and shape [?,1,5]
	 [[{{node serving_default_inputs}}]]
2023-08-22 23:25:01.440464: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-08-22 23:25:01.440478: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-08-22 23:25:01.440596: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpa_q_2qtl
2023-08-22 23:25:01.441473: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-08-22 23:25:01.441482: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (

In [20]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

interpreter.get_signature_list()

{'serving_default': {'inputs': ['inputs'], 'outputs': ['outputs']}}

In [21]:
prediction_fn = interpreter.get_signature_runner("serving_default")

prediction_fn(inputs=x1)

{'outputs': array([[[2.1199878e-13, 9.7491434e-15, 1.9008108e-11, ...,
          4.1016598e-08, 2.6988747e-13, 2.7044811e-13]],
 
        [[6.8798127e-25, 9.4752407e-26, 9.3326709e-21, ...,
          1.0000000e+00, 1.1062211e-27, 9.0354329e-28]],
 
        [[1.1058746e-18, 2.4189967e-15, 1.8075244e-14, ...,
          1.8656558e-10, 1.0710918e-15, 1.0617678e-15]],
 
        ...,
 
        [[3.6503358e-35, 6.8024935e-29, 2.3538978e-26, ...,
          9.6462536e-01, 4.2979137e-21, 1.1815840e-26]],
 
        [[1.4074892e-36, 2.5950507e-26, 1.1894113e-26, ...,
          4.7764579e-09, 2.4879275e-18, 7.6087705e-25]],
 
        [[4.1731929e-40, 1.0042195e-39, 2.4061760e-33, ...,
          1.0000000e+00, 8.8237947e-30, 3.6471239e-35]]], dtype=float32)}